In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import sys
sys.path.append("../src")
from dataLoad import dataLoad, fe_trainLoad

In [5]:
path= "../dataset/"
fe_train_user_seq_log = fe_trainLoad(path)

In [28]:
train = fe_train_user_seq_log.copy()
train_df = train.sort_values(by="timestamp", ascending=False)
train_valid = train_df.drop_duplicates(subset="user", keep="first") 

- test - label처럼, train에서 가장 마지막 click 1개를 valid로 분리합니다.
- eda 결과 유저별로 최소 2개의 click 데이터가 있으므로 2개 이상 떼어낼 시 train에서 유저가 사라지는 이슈가 생깁니다.

In [4]:
# 제거하는 작업
train = train[~train.isin(train_valid)].dropna()

In [5]:
## 데이터 유실 없는지 확인
assert len(train) + len(train_valid) == len(fe_train_user_seq_log)

## valid에는 유저 당 1개의 데이터만 들어가 있음
assert (train_valid.user.value_counts() != 1).sum() == 0

## 각 데이터 셋의 유저군이 동일함을 확인
assert set(train.user) == set(train_valid.user)

- 각 유저에 대해 마지막 click 아이템을 분리했는데, 이때 valid에만 등장하는 아이템은 train에서 학습되지 않아 맞출 수가 없다
- valid에만 있는 아이템은 train에 존재하지 않도록 샘플링

In [7]:
print("train에 있는 user/item만 valid에 남기기")

print(f"valid data size:{len(train_valid):,}")
train_valid = train_valid.merge(train[["user","item_id"]].drop_duplicates(), on=["user","item_id"])
print(f"valid data size:{len(train_valid):,}")

train에 있는 user/item만 valid에 남기기
valid data size:672,116
valid data size:85,675


In [8]:
## unique한 item 개수가 일치한지 확인
train_missed_items2 = set(train_valid.item_id) - set(train.item_id)
assert len(train_missed_items2) == 0

print(f"valid에만 존재하는 아이템 수: {len(train_missed_items2)}개")

valid에만 존재하는 아이템 수: 0개


In [11]:
train_missed_users = set(train_valid.user) - set(train.user)
print(f"valid에만 존재하는 user 수: {len(train_missed_users)}명")

valid에만 존재하는 user 수: 0개


In [12]:
valid_missed_users =  set(train.user) - set(train_valid.user)
print(f"train에만 존재하는 user 수: {len(valid_missed_users)}명")

train에만 존재하는 user 수: 586441명


In [9]:
sample_sumbission = train_valid.copy()
sample_sumbission = sample_sumbission[["user","item_id"]].rename(columns={"item_id":"predicted_list"})
sample_sumbission.item_id = [[]]*len(sample_sumbission)

train_valid = train_valid[["user","item_id"]]

In [11]:
train["user"] = train["user"].astype(int)
train["item_id"] = train["item_id"].astype(int)
train["day_of_week"] = train["day_of_week"].astype(int)
train["days"] = train["days"].astype(int)
train["hour"] = train["hour"].astype(int)
train["weeks"] = train["weeks"].astype(int)
train["cumcount"] = train["cumcount"].astype(int)
train["click_count"] = train["click_count"].astype(int)
train["user_click_count"] = train["user_click_count"].astype(int)

In [11]:
train.to_parquet("../dataset/train.parquet")
train_valid.to_parquet("../dataset/train_valid.parquet")
sample_sumbission.to_parquet("../dataset/sample_sumbission.parquet")

NameError: name 'train' is not defined

In [12]:
item_names_table, train_user_seq_log, test_user_label, test_user_seq_log = dataLoad(path)

In [17]:
final_sumbission = test_user_label.copy()
final_sumbission = final_sumbission[["user","item_id"]]
final_sumbission.item_id = [[]]*len(final_sumbission)
final_sumbission = final_sumbission.rename(columns={"item_id":"predicted_list"})

In [18]:
final_sumbission

,user,predicted_list
0,0,[]
1,1,[]
2,2,[]
3,3,[]
4,4,[]
...,...,...
587481,829386,[]
587482,829390,[]
587483,829393,[]
587484,829397,[]


In [19]:
final_sumbission.to_parquet("../dataset/final_sumbission.parquet")